In [90]:
# Ελεγχος version pyspark .. Θέλουμε το 3.4.1
import pyspark
print(pyspark.__version__)

3.4.1


In [91]:
# add java to path .. δεν μπορουσα να βρω αλλον τροπο να τρέξει η spark.
# (θελουμε την Java 17 επειδη μονο η Java 8 , Java 11 και Java 17 δουλευει με το pyspark)
import os
os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jdk-17"
os.environ["PATH"] += ";C:\\Program Files\\Java\\jdk-17\\bin"

In [92]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from pyspark.sql.types import FloatType
import pandas as pd

In [93]:
# Διαβάζομε τα δεδομένα

tour_occ_ninat = "data/tour_occ_ninat.xlsx"

excel_data = pd.read_excel(tour_occ_ninat, header=8)

In [94]:
# Κάνω το excel σε csv για να το βαλω στην spark , επειδη δεν μου δουλευε το spark-excel

excel_data.to_csv("data/excel_data.csv", index=False)

In [95]:
# Φτιάχνουμε ενα spark session
spark = SparkSession.builder.appName("EurostatAnalysis").getOrCreate()

In [96]:
# Διαβαζουμε το csv με την spark
spark_df = spark.read.option("header", "true").csv("data/excel_data.csv")

In [97]:
# Ενα print για να δουμε οτι διαβασαμε σωστα τα δεδομενα
spark_df.show(truncate=False)

+------------------------------------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|GEO/TIME                                        |2006     |2007     |2008     |2009     |2010     |2011     |2012     |2013     |2014     |2015     |
+------------------------------------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|Belgium                                         |16039090 |16271311 |16360702 |15451017 |16169676 |16723867 |16432646 |16511721 |17068872 |18852087 |
|Bulgaria                                        |11944694 |12006786 |11791454 |9460922  |10547112 |12426723 |13451440 |14370426 |14077798 |13352281 |
|Czech Republic                                  |20090348 |20610186 |19987022 |17746893 |18365947 |19424839 |21793985 |22144896 |22110112 |23286515 |
|Denmark                                         |9453026  |9327579  |8918197  |8299403  |8981

In [98]:
# Αντικαθηστώ το ":" με 0 για να βγουν σωστά οι μέση όροι
# και οποιοδήποτε αλλο string γινεται 0

year_columns = [col for col in spark_df.columns if col.startswith("20")]
for year_col in year_columns:
    spark_df = spark_df.withColumn(
        year_col,
        when(col(year_col).cast(FloatType()).isNotNull(), col(year_col).cast(FloatType())).otherwise(0.0)
    )

In [99]:
# Βλέπω ότι οπου ειχε string τώρα εχει 0

spark_df.show(truncate=False)

+------------------------------------------------+------------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+
|GEO/TIME                                        |2006        |2007       |2008        |2009        |2010        |2011        |2012        |2013        |2014        |2015        |
+------------------------------------------------+------------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+
|Belgium                                         |1.603909E7  |1.6271311E7|1.6360702E7 |1.5451017E7 |1.6169676E7 |1.6723867E7 |1.6432646E7 |1.6511721E7 |1.7068872E7 |1.8852088E7 |
|Bulgaria                                        |1.1944694E7 |1.2006786E7|1.1791454E7 |9460922.0   |1.0547112E7 |1.2426723E7 |1.345144E7  |1.4370426E7 |1.4077798E7 |1.3352281E7 |
|Czech Republic                                  |2.0090348E7 |2.0610186E7|1.9987022E7 |1.7746892E7 

In [100]:
# Ερωτημα 3.1 

year_columns = [str(year) for year in range(2007, 2015)]

In [101]:
# Βρίσκω τον μέσο όρο των στηλών για κάθε χρονο

average_columns = [col(column) for column in year_columns]
average_column = sum(average_columns) / len(average_columns)

In [102]:
# Βάζω το GEO/TIME στην group by για να βρω τον μέσο όρο των χωρών

result = spark_df.select("GEO/TIME", average_column.alias("AverageTourists")) \
    .groupBy("GEO/TIME").avg("AverageTourists")

In [103]:
# Αποτέλεσμα 3.1

result.show(truncate=False)

+------------------------------------------+--------------------+
|GEO/TIME                                  |avg(AverageTourists)|
+------------------------------------------+--------------------+
|Sweden                                    |1.137128175E7       |
|Turkey                                    |2.3396917E7         |
|France                                    |1.18260271E8        |
|Greece                                    |6.0361191E7         |
|Slovakia                                  |4234543.375         |
|Belgium                                   |1.63737265E7        |
|Finland                                   |5450927.375         |
|Malta                                     |7568818.625         |
|Croatia                                   |4.338973875E7       |
|Italy                                     |1.72326804E8        |
|Lithuania                                 |2088391.125         |
|Norway                                    |7966946.375         |
|Spain    